In [21]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/My Drive/Pollen-Adaptation/cyclegan

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/My Drive/Pollen-Adaptation/cyclegan


In [34]:
## install section for Colab

! pip install dominate
! pip install visdom



     |████████████████████████████████| 676 kB 14.9 MB/s 
     |████████████████████████████████| 54 kB 2.9 MB/s 
  Created wheel for visdom: filename=visdom-0.1.8.9-py3-none-any.whl size=655250 sha256=2139c32a7b8f8403fabf6d9e06ee95dcf6440e70c8f6b386826b94c3883ef36c
  Stored in directory: /root/.cache/pip/wheels/2d/d1/9b/cde923274eac9cbb6ff0d8c7c72fe30a3da9095a38fd50bbf1
  Created wheel for torchfile: filename=torchfile-0.1.0-py3-none-any.whl size=5709 sha256=4d0ec2c74562a9eed070fa92fcfacba2c6292a7f0ce597c1a96ee3caec40cb04
  Stored in directory: /root/.cache/pip/wheels/ac/5c/3a/a80e1c65880945c71fd833408cd1e9a8cb7e2f8f37620bb75b
Successfully built visdom torchfile


In [35]:
import time
import argparse
import torch
from data import CreateDataLoader
from models import create_model
from util.visualizer import Visualizer
from options.train_options import TrainOptions
torch.cuda.is_available()

True

In [36]:
# python train.py --name cycada_svhn2mnist_noIdentity \
#     --resize_or_crop=None \
#     --loadSize=32 --fineSize=32 --which_model_netD n_layers --n_layers_D 3 \
#     --model cycle_gan_semantic \
#     --lambda_A 1 --lambda_B 1 --lambda_identity 0 \
#     --no_flip --batchSize 100 \
#     --dataset_mode mnist_svhn --dataroot /x/jhoffman/ \
#     --which_direction BtoA

In [37]:
### Loading parameters configuration
opt = TrainOptions()
opt.initialize()
trainOPT = opt.parser.parse_args(args=[ '--name', 'cycada_svhn2mnist_noIdentity',
                                       '--resize_or_crop', 'None',
                                       '--loadSize', '32',
                                       '--fineSize', '32',
                                       '--which_model_netD', 'n_layers',
                                       '--n_layers_D', '3',
                                       '--model', 'cycle_gan_semantic',
                                       '--lambda_A', '1',
                                       '--lambda_B', '1',
                                       '--lambda_identity', '0',
                                       '--no_flip',
                                       '--batchSize', '100',
                                       '--dataset_mode', 'mnist_svhn',
                                       '--which_direction', 'BtoA',
                                       '--dataroot', '/content/drive/My Drive/Pollen-Adaptation/cyclegan/dataset',
                                        ])
trainOPT.isTrain = True
trainOPT.gpu_ids = [int(trainOPT.gpu_ids)]
print(trainOPT.gpu_ids)


[0]


In [38]:
##  Loading Data

data_loader = CreateDataLoader(trainOPT)

dataset = data_loader.load_data()
dataset_size = len(data_loader)
print('#training images = %d' % dataset_size)


CustomDatasetDataLoader
dataset [MnistSvhnDataset] was created
Namespace(batchSize=100, beta1=0.5, checkpoints_dir='./checkpoints', continue_train=False, dataroot='/content/drive/My Drive/Pollen-Adaptation/cyclegan/dataset', dataset_mode='mnist_svhn', display_freq=400, display_id=1, display_ncols=4, display_port=8097, display_server='http://localhost', display_winsize=256, epoch_count=1, fineSize=32, gpu_ids=[0], init_type='normal', input_nc=3, isTrain=True, lambda_A=1.0, lambda_B=1.0, lambda_identity=0.0, loadSize=32, lr=0.0002, lr_decay_iters=50, lr_policy='lambda', max_dataset_size=inf, model='cycle_gan_semantic', nThreads=4, n_layers_D=3, name='cycada_svhn2mnist_noIdentity', ndf=64, ngf=64, niter=100, niter_decay=100, no_dropout=False, no_flip=True, no_html=False, no_lsgan=False, norm='instance', output_nc=3, phase='train', pool_size=50, print_freq=100, resize_or_crop='None', save_epoch_freq=5, save_latest_freq=5000, serial_batches=False, suffix='', update_html_freq=1000, verbose=F

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [44]:
## initialize model

model = create_model(trainOPT)
model.setup(trainOPT)


initialize network with normal
initialize network with normal
initialize network with normal
initialize network with normal
initialize network with normal
model [CycleGANModel] was created
---------- Networks initialized -------------
[Network G_A] Total number of parameters : 11.378 M
[Network G_B] Total number of parameters : 11.378 M
[Network D_A] Total number of parameters : 2.765 M
[Network D_B] Total number of parameters : 2.765 M
[Network CLS] Total number of parameters : 0.682 M
-----------------------------------------------


In [46]:
## visualizer
visualizer = Visualizer(trainOPT)

Setting up a new session...


create web directory ./checkpoints/cycada_svhn2mnist_noIdentity/web...


In [ ]:

total_steps = 0

for epoch in range(trainOPT.epoch_count, trainOPT.niter + trainOPT.niter_decay + 1):
    epoch_start_time = time.time()
    iter_data_time = time.time()
    epoch_iter = 0

    for i, data in enumerate(dataset):
        iter_start_time = time.time()
        if total_steps % trainOPT.print_freq == 0:
            t_data = iter_start_time - iter_data_time
        visualizer.reset()
        total_steps += trainOPT.batchSize
        epoch_iter += trainOPT.batchSize
        model.set_input(data)
        model.optimize_parameters()

        if total_steps % trainOPT.display_freq == 0:
            save_result = total_steps % trainOPT.update_html_freq == 0
            visualizer.display_current_results(model.get_current_visuals(), epoch, save_result)

        if total_steps % trainOPT.print_freq == 0:
            losses = model.get_current_losses()
            t = (time.time() - iter_start_time) / trainOPT.batchSize
            visualizer.print_current_losses(epoch, epoch_iter, losses, t, t_data)
            if trainOPT.display_id > 0:
                visualizer.plot_current_losses(epoch, float(epoch_iter) / dataset_size, trainOPT, losses)

        if total_steps % trainOPT.save_latest_freq == 0:
            print('saving the latest model (epoch %d, total_steps %d)' %
                  (epoch, total_steps))
            model.save_networks('latest')

        iter_data_time = time.time()
    if epoch % trainOPT.save_epoch_freq == 0:
        print('saving the model at the end of epoch %d, iters %d' %
              (epoch, total_steps))
        model.save_networks('latest')
        model.save_networks(epoch)

    print('End of epoch %d / %d \t Time Taken: %d sec' %
          (epoch, trainOPT.niter + trainOPT.niter_decay, time.time() - epoch_start_time))
    model.update_learning_rate()

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


(epoch: 1, iters: 100, time: 0.011, data: 0.301) D_A: 1.263 G_A: 4.103 cycle_A: 0.594 idt_A: 0.000 D_B: 1.111 G_B: 1.266 cycle_B: 0.990 idt_B: 0.000 sem_AB: 0.000 sem_BA: 0.000 CLS: 2.309 
(epoch: 1, iters: 200, time: 0.004, data: 0.000) D_A: 6.719 G_A: 2.298 cycle_A: 0.578 idt_A: 0.000 D_B: 3.786 G_B: 2.210 cycle_B: 0.990 idt_B: 0.000 sem_AB: 0.000 sem_BA: 0.000 CLS: 2.276 
(epoch: 1, iters: 300, time: 0.004, data: 0.003) D_A: 0.704 G_A: 3.434 cycle_A: 0.569 idt_A: 0.000 D_B: 1.012 G_B: 2.928 cycle_B: 0.976 idt_B: 0.000 sem_AB: 0.000 sem_BA: 0.000 CLS: 2.185 
(epoch: 1, iters: 400, time: 0.005, data: 0.003) D_A: 1.939 G_A: 6.674 cycle_A: 0.557 idt_A: 0.000 D_B: 0.884 G_B: 2.195 cycle_B: 0.919 idt_B: 0.000 sem_AB: 0.000 sem_BA: 0.000 CLS: 2.289 
(epoch: 1, iters: 500, time: 0.004, data: 0.002) D_A: 0.811 G_A: 2.920 cycle_A: 0.542 idt_A: 0.000 D_B: 0.836 G_B: 0.642 cycle_B: 0.857 idt_B: 0.000 sem_AB: 0.000 sem_BA: 0.000 CLS: 2.332 
(epoch: 1, iters: 600, time: 0.004, data: 0.000) D_A: 0